In [ ]:
#!pip install piml
!pip install pandas-profiling
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# EDA

In [ ]:

# mount google drive
from google.colab import drive
drive.mount("/content/drive")

#import data from my google drive folder
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/6010 health/data/LengthOfStay.csv")
df.describe(include='all')
data=df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# importing packages
import pandas as pd
from pandas_profiling import ProfileReport

# forming ProfileReport and save
# as output.html file
profile = ProfileReport(df)
profile

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# #!pip install autoviz
# from autoviz.AutoViz_Class import AutoViz_Class
# AV = AutoViz_Class()
# df_av = AV.AutoViz("/content/drive/MyDrive/6010 health/data/LengthOfStay.csv")

Feature Engineering
## Feature Engineering:
## 1- Standardization: hematocrit, neutrophils, sodium, glucose, bloodureanitro, creatinine, bmi, pulse, respiration.
## 2- Number of preidentified medical conditions: number_of_issues.



In [ ]:
from sklearn.preprocessing import OneHotEncoder
# Create an instance of OneHotEncoder
encoder = OneHotEncoder()

# Assuming 'data' is your DataFrame and 'gender' is the column you want to one-hot encode
gender_encoded = encoder.fit_transform(data[['gender']])

# Convert the result to a DataFrame and specify column names
gender_encoded_df = pd.DataFrame(gender_encoded.toarray(), columns=encoder.get_feature_names_out(['gender']))

# Make sure the indices match
gender_encoded_df.index = data.index

# Concatenate the one-hot encoded DataFrame with the original DataFrame
data = pd.concat([data, gender_encoded_df], axis=1)

# Drop the original 'gender' column
data.drop(columns=['gender'], inplace=True)

# Drop the original 'gender' column
#data.drop(columns=['gender'], inplace=True)

In [ ]:
data.head()

,eid,vdate,rcount,dialysisrenalendstage,asthma,irondef,pneum,substancedependence,psychologicaldisordermajor,depress,...,creatinine,bmi,pulse,respiration,secondarydiagnosisnonicd9,discharged,facid,lengthofstay,gender_F,gender_M
0,1,8/29/2012,0,0,0,0,0,0,0,0,...,1.390722,30.432418,96,6.5,4,9/1/2012,B,3,1.0,0.0
1,2,5/26/2012,5+,0,0,0,0,0,0,0,...,0.943164,28.460516,61,6.5,1,6/2/2012,A,7,1.0,0.0
2,3,9/22/2012,1,0,0,0,0,0,0,0,...,1.065750,28.843812,64,6.5,2,9/25/2012,B,3,1.0,0.0
3,4,8/9/2012,0,0,0,0,0,0,0,0,...,0.906862,27.959007,76,6.5,1,8/10/2012,A,1,1.0,0.0
4,5,12/20/2012,0,0,0,0,1,0,1,0,...,1.242854,30.258927,67,5.6,2,12/24/2012,E,4,1.0,0.0


In [ ]:
from sklearn.preprocessing import StandardScaler
# Standardize the specified columns
scaler = StandardScaler()
columns_to_standardize = ['hematocrit', 'neutrophils', 'sodium', 'glucose', 'bloodureanitro', 'creatinine', 'bmi', 'pulse', 'respiration']
data[columns_to_standardize] = scaler.fit_transform(data[columns_to_standardize])


In [ ]:
#Creating a variables that represents the number of issues a patient had.
# This helps reduce the dimensions for our model

data["number_of_issues"] = data[['hemo', 'dialysisrenalendstage', 'asthma', 'irondef', 'pneum', 'substancedependence',
                                  'psychologicaldisordermajor', 'depress', 'psychother', 'fibrosisandother', 'malnutrition']].sum(axis=1)

#sums up the values in the specified columns and assigns the result to 'number_of_issues'


In [ ]:
data['stay_long'] = (data['lengthofstay'] >= 7).astype(float)

In [ ]:
columns_to_drop = ['hemo', 'dialysisrenalendstage', 'asthma', 'irondef', 'pneum', 'substancedependence',
                                  'psychologicaldisordermajor', 'depress', 'psychother', 'fibrosisandother', 'malnutrition','rcount','eid']
data1 = data.drop(columns=columns_to_drop)

In [ ]:
# Splitting the data in to our predictor variables and target variable (Length of stay greater than 7 days)
predictor_variables = ['hematocrit', 'neutrophils', 'sodium','glucose',
                      'bloodureanitro', 'creatinine', 'bmi', 'pulse', 'respiration','gender_F' ,'gender_M','number_of_issues']
X = data1[predictor_variables]
Y = data1['stay_long']

In [ ]:
data1.head()

,vdate,hematocrit,neutrophils,sodium,glucose,bloodureanitro,creatinine,bmi,pulse,respiration,secondarydiagnosisnonicd9,discharged,facid,lengthofstay,gender_F,gender_M,number_of_issues,stay_long
0,8/29/2012,-0.234247,0.751441,0.823340,1.684186,-0.161915,1.454965,0.312741,1.936991,0.010963,4,9/1/2012,B,3,1.0,0.0,0,0.0
1,5/26/2012,-1.464694,-1.135314,-0.386613,-1.596543,-0.470738,-0.779911,-0.671360,-1.068721,0.010963,1,6/2/2012,A,7,1.0,0.0,0,1.0
2,9/22/2012,-1.760001,-0.238638,-1.611147,-0.381186,-0.161915,-0.167779,-0.480071,-0.811089,0.010963,2,9/25/2012,B,3,1.0,0.0,0,0.0
3,8/9/2012,-0.037375,-0.145235,0.367584,0.713958,-0.161915,-0.961187,-0.921644,0.219441,0.010963,1,8/10/2012,A,1,1.0,0.0,0,0.0
4,12/20/2012,-1.415476,-0.210617,0.247842,-1.569599,-0.200518,0.716587,0.226159,-0.553456,-1.572234,2,12/24/2012,E,4,1.0,0.0,2,0.0


## Splitting data into training and testing set

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

## Model Training and Evaluation

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)


y_pred = rf_model.predict(X_test)

from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 0.2372157699610846


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
#Create a GradientBoostingRegressor instance
gb_regressor = GradientBoostingRegressor(n_estimators=150, learning_rate=0.1, max_depth=3, random_state=42)

# Fit the model to the training data
gb_regressor.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = gb_regressor.predict(X_test)
mae1 = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae1}")

In [ ]:
hyperparam_impact

In [ ]:
# Initialize lists to store the results
train_scores = []
test_scores = []
learning_r_values = [0.01, 0.2, 0.3]
# Loop through the 'n_estimators' values and evaluate the model
for n_estimators in learning_r_values:
    # Create and train the model with the current 'n_estimators' value
    model = GradientBoostingRegressor(learning_rate=n_estimators, random_state=42)
    model.fit(X_train, y_train)

    # Evaluate the model on the training set
    train_score = model.score(X_train, y_train)
    train_scores.append(train_score)

    # Evaluate the model on the testing set
    test_score = model.score(X_test, y_test)
    test_scores.append(test_score)

In [ ]:
# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(learning_r_values, train_scores, label='Train Score', marker='o')
plt.plot(learning_r_values, test_scores, label='Test Score', marker='o')
plt.xlabel('n_estimators')
plt.ylabel('R-squared Score')
plt.title('Sensitivity Analysis for n_estimators')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
gb_regressor1= GradientBoostingRegressor(n_estimators=150, learning_rate=0.2, max_depth=3, random_state=42)

# Fit the model to the training data
gb_regressor1.fit(X_train, y_train)
# Make predictions on the testing data
y_pred1 = gb_regressor1.predict(X_test)
mae2 = mean_absolute_error(y_test, y_pred1)
print(f"Mean Absolute Error: {mae2}")

In [ ]:
profile1 = ProfileReport(data1)
profile1

NameError: ignored